# Import Python Libraries

In [1]:
import pandas as pd
from sklearn.compose import ColumnTransformer, make_column_selector, make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
import numpy as np
from joblib import dump
from sklearn.linear_model import LogisticRegression
from sklearn import set_config
import pandas as pd
from dotenv import dotenv_values
import os

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# 1. Download and Enable Db2 Magic Commands Extension for Jupyter Notebook

In [3]:
# Enable Db2 Magic Commands Extensions for Jupyter Notebook
if not os.path.isfile('db2.ipynb'):
    os.system('wget https://raw.githubusercontent.com/IBM/db2-jupyter/master/db2.ipynb')
%run db2.ipynb

Db2 Extensions Loaded. Version: 2024-09-16


## Load db2 connection config from the `db2con.env` file

In [4]:
db2creds = dotenv_values('db2con.env')
%sql CONNECT CREDENTIALS db2creds

Connection successful. db2ai @ localhost 


# Load ML dataset from Db2

In [5]:
query = %sql SELECT * FROM FLIGHTS_TRAIN
df = pd.DataFrame(query)

In [6]:
cols_show = ['MONTH','DAYOFWEEK', 'UNIQUECARRIER', 'ORIGIN', 'DEST', 'DEPDELAY', 'FLIGHTSTATUS']

print('shape of the dataset: ', df.shape)

print('sample rows from the dataset:')
df[cols_show].sample(5)

shape of the dataset:  (160000, 19)
sample rows from the dataset:


,MONTH,DAYOFWEEK,UNIQUECARRIER,ORIGIN,DEST,DEPDELAY,FLIGHTSTATUS
128027,7,5,WN,PHX,LAX,44.0,1
11276,4,5,B6,DTW,BOS,0.0,0
21613,2,2,DL,SMF,ATL,-7.0,0
37743,3,1,UA,BOS,EWR,0.0,0
115761,8,7,WN,RDU,PHX,-2.0,0


# Build and Train a Classification Model using Scikit-Learn

In [7]:
# Randomly split the dataset into 2 datasets: train and test sets. The test set has 20% of the original samples. 
# The remaining 80% of the samples remain with the train set. 
df_test = df.sample(frac=0.20)
df_train = df.drop(df_test.index)

# select the subset of columns as the input features for the ML model
input_cols = ['YEAR','QUARTER', 'MONTH',
                      'DAYOFMONTH', 'DAYOFWEEK','UNIQUECARRIER',
                      'ORIGIN', 'DEST', 'CRSDEPTIME',
                      'DEPDELAY', 'DEPDEL15','TAXIOUT','WHEELSOFF',
                      'CRSARRTIME', 'CRSELAPSEDTIME', 'AIRTIME', 'DISTANCEGROUP']

# select the class label, the target column, for the classification model
target = ['FLIGHTSTATUS']

X = df_train[input_cols]
y = df_train[target].astype('int')

# define the strategy to fill in missing values in the numeric columns
num_pipeline = make_pipeline(SimpleImputer(strategy='constant', fill_value=0), 
                            MaxAbsScaler())

# define the strategy to fill in missing values in the categorical columns
cat_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'),
                            OneHotEncoder(handle_unknown='ignore'))

# combine the previous 2 pipelines into a data preproessing pipeline. 

preprocessing = make_column_transformer(
    (num_pipeline, make_column_selector(dtype_include=np.number)),
    (cat_pipeline, make_column_selector(dtype_include='object'))
)

# create a final pipeline by chaining data preprocessing and a learning algorithm, `LogisticRegression`

pipe_lr = make_pipeline(preprocessing, 
                       LogisticRegression(random_state=1,
                                         solver='lbfgs'))

# train the model using the training set features and class labels
pipe_lr.fit(X, y)

# evaluate the trained model using the test set
X_test = df_test[input_cols]
y_test = df_test[target].astype('int')

predictions = pipe_lr.predict(X_test)

# compute the print the model accuracy

print('Accuracy: ', pipe_lr.score(X_test, y_test) * 100)

# retrain the model using the complete dataset 

pipe_lr.fit(df[input_cols], df[target].astype('int'))

Accuracy:  87.6375


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant')),
                                                                  ('maxabsscaler',
                                                                   MaxAbsScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f74cc9b98d0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f74ccccacd0>)])),
                ('logisticregression', LogisticRegression(random_state=1))])

# Serialize and export the trained model pipeline

In [14]:
directory = '/home/db2inst1/UDF'
os.makedirs(directory, exist_ok=True)

joblib_file = 'myudf.joblib'
joblib_file_path = os.path.join(directory, joblib_file)

In [15]:
dump(pipe_lr,joblib_file_path)

['/home/db2inst1/UDF/myudf.joblib']

# Write UDF to a .py file

In [10]:
udf_file = 'myudf.py'
udf_file_path = os.path.join(directory, udf_file)

In [11]:
%%writefile {udf_file_path}

################
### IMPORTS ###
###############
import nzae

import pandas as pd
from joblib import load

ml_model_features = ['YEAR', 'QUARTER', 'MONTH', 'DAYOFMONTH', 'DAYOFWEEK', 'UNIQUECARRIER', 'ORIGIN', 'DEST', 'CRSDEPTIME', 'DEPDELAY', 'DEPDEL15', 'TAXIOUT', 'WHEELSOFF', 'CRSARRTIME', 'CRSELAPSEDTIME', 'AIRTIME', 'DISTANCEGROUP']

class full_pipeline(nzae.Ae):
    def _runUdtf(self):
        #####################a
        ### INITIALIZATON ###
        #####################
    
        trained_pipeline = load('/home/db2inst1/UDF/myudf.joblib')
        
        #######################
        ### DATA COLLECTION ###
        #######################
        # Collect rows into a single batch
        rownum = 0
        row_list = []
        for row in self:
            if (rownum==0):
                # Grab batchsize from first element value (select count (*))
                batchsize=row[0] 
            
            # Collect everything but first element (which is select count(*))
            row_list.append(row[1:])
            rownum = rownum+1
            if rownum==batchsize:
                ##############################
                ### MODEL SCORING & OUTPUT ###
                ##############################
                
                # Collect data into a Pandas dataframe for scoring
                data=pd.DataFrame(row_list,columns=ml_model_features)
                
                # Call our trained pipeline to transform the data and make predictions
                predictions = trained_pipeline.predict(data)
                
                # Output the columns along with the corresponding prediction
                for x in range(predictions.shape[0]):
                    outputs = []
                    for i in row_list[x]:
                        outputs.append(i)
                    if predictions.dtype.kind=='i':
                        outputs.append(int(predictions[x]))
                    else:
                        outputs.append(float(predictions[x]))
                    self.output(outputs)

                #Reset rownum and row_list for next batch
                row_list=[]
                rownum=0
        self.done()
full_pipeline.run()

Writing /home/db2inst1/UDF/myudf.py


# Register the UDF on Db2

In [12]:
%sql DROP FUNCTION MYUDF

Command completed.


In [13]:
%%sql

CREATE OR REPLACE FUNCTION MYUDF(
    INTEGER,
    INTEGER,
    INTEGER,
    INTEGER,
    INTEGER,
    INTEGER,
    VARCHAR(50),
    VARCHAR(50),
    VARCHAR(50),
    INTEGER,
    REAL,
    REAL,
    INTEGER,
    INTEGER,
    INTEGER,
    INTEGER,
    INTEGER,
    INTEGER
) 
RETURNS TABLE (
    "YEAR" INTEGER,
    "QUARTER" INTEGER,
    "MONTH" INTEGER,
    "DAYOFMONTH" INTEGER,
    "DAYOFWEEK" INTEGER,
    "UNIQUECARRIER" VARCHAR(50),
    "ORIGIN" VARCHAR(50),
    "DEST" VARCHAR(50),
    "CRSDEPTIME" INTEGER,
    "DEPDELAY" REAL,
    "DEPDEL15" REAL,
    "TAXIOUT" INTEGER,
    "WHEELSOFF" INTEGER,
    "CRSARRTIME" INTEGER,
    "CRSELAPSEDTIME" INTEGER,
    "AIRTIME" INTEGER,
    "DISTANCEGROUP" INTEGER,
    "FLIGHTSTATUS_PREDICTION" INTEGER
)
LANGUAGE PYTHON 
PARAMETER STYLE NPSGENERIC  
FENCED  
NOT THREADSAFE  
NO FINAL CALL  
DISALLOW PARALLEL  
NO DBINFO  
DETERMINISTIC 
NO EXTERNAL ACTION 
CALLED ON NULL INPUT  
NO SQL 
EXTERNAL NAME '/home/db2inst1/UDF/myudf.py'

Command completed.


# Generate Predictions using this UDF

In [16]:
%%sql
SELECT f.*
FROM FLIGHTS_TEST i, 
     TABLE(
         MYUDF(
             (SELECT COUNT(*) FROM FLIGHTS_TEST),
             i."YEAR",
             i."QUARTER",
             i."MONTH",
             i."DAYOFMONTH",
             i."DAYOFWEEK",
             i."UNIQUECARRIER",
             i."ORIGIN",
             i."DEST",
             i."CRSDEPTIME",
             i."DEPDELAY",
             i."DEPDEL15",
             i."TAXIOUT",
             i."WHEELSOFF",
             i."CRSARRTIME",
             i."CRSELAPSEDTIME",
             i."AIRTIME",
             i."DISTANCEGROUP"
         )
     ) f

,YEAR,QUARTER,MONTH,DAYOFMONTH,DAYOFWEEK,UNIQUECARRIER,ORIGIN,DEST,CRSDEPTIME,DEPDELAY,DEPDEL15,TAXIOUT,WHEELSOFF,CRSARRTIME,CRSELAPSEDTIME,AIRTIME,DISTANCEGROUP,FLIGHTSTATUS_PREDICTION
0,2016,4,10,16,7,B6,BOS,ORD,1642,57.0,1.0,18,1757,1837,175,127,4,1
1,2011,3,9,19,1,WN,PHX,SFO,750,-4.0,0.0,12,758,1000,130,98,3,0
2,2011,1,2,10,4,WN,PIT,PHL,1710,-3.0,0.0,8,1715,1830,80,47,2,0
3,2012,4,11,12,1,B6,BUF,JFK,1432,327.0,1.0,7,2006,1600,88,63,2,1
4,2014,3,7,29,2,WN,LGA,BNA,855,-4.0,0.0,38,929,1025,150,112,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,2012,3,7,31,2,MQ,BRO,DFW,1155,-8.0,0.0,8,1155,1335,100,78,2,0
39996,2018,2,5,20,7,WN,SAT,DAL,605,5.0,0.0,13,623,710,65,51,1,0
39997,2013,3,9,5,4,EV,BTR,IAH,1136,-14.0,0.0,7,1129,1245,69,52,2,0
39998,2011,1,3,3,4,DL,SEA,MSP,1230,0.0,0.0,17,1247,1746,196,161,6,0
